In [ ]:
import requests
import json
from tqdm import tqdm
import time
import os
import random

URLs

- http://13.215.146.238/api/api.php/select_province_and_zone

This URL will return a list of all the provinces and election zones in the country.

- http://13.215.146.238/api/api.php/select_election_zone_candidates?province=71&zoneNumber=2

This URL will return the details of all the election candidates in the province with the ID `71` and zone number `2`.

- http://13.215.146.238/api/api.php/select_election_candidate?candidateId=e8ce23553399625dcf33b029d29c2084

This URL will return the details of the election candidate with the ID `e8ce23553399625dcf33b029d29c2084`.


- http://13.215.146.238/images/candidate/thumb/71/2/7f2ebf81374ec54fd727f65d882c88d3.jpg

This URL will return a thumbnail image of the election candidate with the ID `e8ce23553399625dcf33b029d29c2084`.

In [ ]:
def getter(slug, params={}):
    ep = 'http://13.215.146.238/api/api.php/' + slug
    resp = requests.get(ep, params=params)
    data = resp.json()
    if 'MessageCode' in data.keys() and data['MessageCode'] == '00000':
        return data['Result']
    if 'error_code' in data.keys() and data['error_code'] == 0:
        return data['data_list']
    return []

In [ ]:
def get_candidate_info(candidate_id):
    return getter('select_election_candidate',
                  {'candidateId': candidate_id})

In [ ]:
get_candidate_info('e8ce23553399625dcf33b029d29c2084')

In [ ]:
def get_candidates_in_district(prov_code, elect_dist_number):
    return getter('select_election_zone_candidates',
                  {'province': prov_code, 'zoneNumber': prov_code})

In [ ]:
resp = requests.get('http://13.215.146.238/images/candidate/thumb/71/2/7f2ebf81374ec54fd727f65d882c88d3.jpg')

In [ ]:
province_list = getter('select_province_and_zone')

In [ ]:
def candidate_list_from_api()
    candidate_list = []
    for province in tqdm(province_list):
        dist_nums = len(province['zone_list'])
        prov_code = province['province_code']
        for dist_num in range(1, dist_nums+1):
            candidates = get_candidates_in_district(prov_code, dist_nums)
            for candidate in candidates:
                candidate['ProvinceName'] = province['province_name']
                candidate['ProvinceCode'] = prov_code
                candidate['ZoneNumber'] = dist_num
            candidate_list += candidates
    return candidate_list

In [ ]:
# with open('smartvote-candidates.json','w') as fp:
#     json.dump(candidate_list, fp, ensure_ascii=False)

In [ ]:
def save_candidate_file(candidate_list):
    with open('smartvote-candidates.json','w') as fp:
        json.dump(candidate_list, fp, ensure_ascii=False)

In [ ]:
for i,candidate in tqdm(enumerate(candidate_list)):
    if 'Age' in candidate.keys(): continue
    candidate_id = candidate['CandidateId']
    new_info = get_candidate_info(candidate_id)
    candidate.update(new_info)
    
    # sleep
    time.sleep(random.randrange(10,500)/1000)
    
    if random.random() < 0.01:
        time.sleep(1)
    if i % 100 == 0 or i == len(candidate_list) - 1:
        save_candidate_file(candidate_list)

In [ ]:
save_candidate_file(candidate_list)

In [ ]:
import pandas as pd

In [ ]:
candidate_df = pd.DataFrame(candidate_list).drop(columns='Id')

In [ ]:
candidate_df